<a href="https://colab.research.google.com/github/nonotoy/yubimoji/blob/main/yubimoji_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install tensorflow==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 44.2 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.4:
      Successfully uninstalled gast-0.5.4
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.0
    Uninstalling google-auth-oauthlib-1.2.0:
      Successfully u

In [4]:
import csv
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

dataset = '/content/drive/MyDrive/Colab Notebooks/JSL/point_history_normalised.csv'

In [5]:
# モデルのパラメータ
n_features = 40  # ランドマークの数
n_sequence = 30  # シーケンス長 (調整が必要)
n_classes = 76 # 出力クラス (指文字) の数

# 2列目以降を学習データとする
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, n_features+1))) # (20730, 40)

# 1列目を正解ラベルとする
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

# ラベルをワンホットエンコードする
y_dataset = to_categorical(y_dataset, num_classes=n_classes) # (20730, 76)

# print(X_dataset.shape[0] // n_sequence)  # 691

# サンプル数を n_sequence で割り切れるように調整
total_samples = len(X_dataset)
max_samples = total_samples - total_samples % n_sequence # 20,730 = 20,730 - 0

# Reshape
X_dataset = X_dataset[:max_samples]
y_dataset = y_dataset[:max_samples]
X_dataset = X_dataset.reshape(-1, n_sequence, n_features) # (691, 30, 40)
y_dataset = y_dataset.reshape(-1, n_sequence, n_classes) # (691, 30, 76)

## LSTMモデル

### モデル構築

In [6]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D, LeakyReLU
from keras.regularizers import l2

model_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/model/gesture_classifier.hdf5'

X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.8, random_state=42)
y_train = y_train[:, -1, :]
y_test = y_test[:, -1, :]

# モデル
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_sequence, n_features)),
    MaxPooling1D(pool_size=2),
    LSTM(512, return_sequences=False, kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(256, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(128, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(64, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(n_classes, activation='softmax')
])

# モデルのコンパイル
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path, verbose=1, save_weights_only=False)

# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

# モデルのサマリー表示
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 28, 64)            7744      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 14, 64)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 512)               1181696   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                        

In [7]:
# 訓練
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
5/5 [==============================] - ETA: 0s - loss: 5.0347 - accuracy: 0.0254
Epoch 1: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/gesture_classifier.hdf5
5/5 [==============================] - 5s 289ms/step - loss: 5.0347 - accuracy: 0.0254 - val_loss: 4.9275 - val_accuracy: 0.0504
Epoch 2/1000
5/5 [==============================] - ETA: 0s - loss: 4.8632 - accuracy: 0.0562
Epoch 2: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/gesture_classifier.hdf5
5/5 [==============================] - 1s 142ms/step - loss: 4.8632 - accuracy: 0.0562 - val_loss: 4.7889 - val_accuracy: 0.0791
Epoch 3/1000
5/5 [==============================] - ETA: 0s - loss: 4.7053 - accuracy: 0.0797
Epoch 3: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/gesture_classifier.hdf5
5/5 [==============================] - 1s 137ms/step - loss: 4.7053 - accuracy: 0.0797 - val_loss: 4.6641 - val_accuracy: 0.0576
Epoch 4/1000
5/5 [===================

In [8]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# LSTM 256, Dropout 0.3, Dense1 64, Dense2 32, Batch 128, Adam
# ReLU Accuracy: 32.95%
# LeakyReLU Accuracy: 39.31%
# ELU Accuracy: 35.84%

# LSTM 128, Dropout 0.3, Dense1 64, Dense2 32, Batch 128, Adam
# LeakyReLU Accuracy: 33.53%

# LSTM 512, Dropout 0.3, Dense1 64, Dense2 32, Batch 128, Adam
# LeakyReLU Accuracy: 42.20%

# LSTM 512, Dropout 0.2, Dense1 64, Dense2 32, Batch 128, Adam
# LeakyReLU Accuracy: 46.82%

# LSTM 512, Dropout 0.2, Dense1 128, Dense2 64, Batch 128, Adam
# LeakyReLU Accuracy: 49.71%

# LSTM 512, Dropout 0.2, Dense1 128, Dense2 64, Batch 128, NAdam
# LeakyReLU Accuracy: 50.87%

# Conv1D (ReLU) Kernel=3, LSTM 512, Dropout 0.2, Dense1 128, Dense2 64, Batch 128, NAdam
# LeakyReLU Accuracy: 56.07%
# ReLU Accuracy: 49.71%

# Conv1D (LeakyReLU), LSTM 512, Dropout 0.2, Dense1 128, Dense2 64, Batch 128, NAdam
# LeakyReLU Accuracy: 54.34%

# Conv1D (ReLU) Kernel=5, LSTM 512, Dropout 0.2, Dense1 128, Dense2 64, Batch 128, NAdam
# LeakyReLU Accuracy: 48.55%

# Conv1D (ReLU) Kernel=4, LSTM 512, Dropout 0.2, Dense1 128, Dense2 64, Batch 128, NAdam
# LeakyReLU Accuracy: 52.02%

# Conv1D (ReLU) Kernel=3, LSTM 512, Dropout 0.2, Dense1 128, Dense2 64, Batch 128, NAdam
# LeakyReLU Accuracy: 56.07%
# Early Stoppingは変えない方よさそう、Nadam

# Conv1D (ReLU) Kernel=3, LSTM 512, Dropout 0.2, Dense1 128, Dense2 64, Batch 128, NAdam, L2正規化
# LeakyReLU Accuracy: 57.23%

# Conv1D (ReLU) Kernel=3, LSTM 512, Dropout 0.2, Dense1 256, Dense2 128, Dense3 64, Batch 128, NAdam, L2正規化
# LeakyReLU Accuracy: 58.38%


Accuracy: 55.40%


In [9]:
# 推論テスト
# predict_result = model.predict(np.array([X_test[0]]))

# 推論モデルとして保存
model.save(model_save_path, include_optimizer=False)

### 量子化

In [10]:
# 推論モデルを読み込み
import tensorflow as tf
from tensorflow import keras

model = tf.keras.models.load_model(model_save_path)
print(model.input_shape)

(None, 30, 40)


In [11]:
# モデルを変換(量子化)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# TensorFlow Liteでサポートされていない操作を含むモデルに対応するための設定
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,tf.lite.OpsSet.SELECT_TF_OPS]

converter._experimental_lower_tensor_list_ops = False

tflite_quantized_model = converter.convert()
tflite_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/model/keypoint_classifier.tflite'

open(tflite_save_path, 'wb').write(tflite_quantized_model)

1397064

In [12]:
# 推論テスト
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

# 入出力テンソルを取得
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details[0]['shape'])

# 推論実施の前に、入力データを適切な形状にリシェイプ
input_data = np.array([X_dataset[0]]) #.reshape(1, TIME_STEPS, DIMENSION)

print(input_data.shape)

[ 1 30 40]
(1, 30, 40)


In [13]:
# 推論実施
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[2.5682239e-08 5.7404926e-05 4.8790056e-09 8.6089074e-07 3.3082029e-05
 9.3010684e-08 5.2445532e-05 1.9723300e-06 5.8425641e-05 1.2767079e-04
 2.1821474e-05 2.3752381e-07 1.5148571e-05 3.6736008e-08 1.0552940e-09
 3.3968845e-06 4.3938644e-03 8.1095487e-01 1.4688464e-05 3.7573809e-06
 4.3697441e-05 1.3225562e-07 3.1281090e-08 1.8350585e-05 3.2421058e-06
 1.0234660e-09 2.1117960e-06 8.1920439e-07 8.3569603e-06 6.1915978e-07
 2.3587090e-06 1.3187792e-05 1.1291385e-05 7.9280051e-04 2.1890205e-06
 4.7657641e-06 5.3782687e-06 6.8831632e-06 3.8925783e-08 7.2758428e-07
 1.8507150e-08 7.3531503e-08 3.5059446e-07 1.4631429e-08 1.0119501e-04
 3.3958685e-07 2.8449983e-07 3.4895950e-04 7.2298976e-06 2.3664157e-05
 1.4405890e-03 8.7092940e-06 4.9275491e-07 9.9417832e-08 3.7216114e-06
 8.9349804e-08 4.3058576e-06 2.7416658e-03 1.2467092e-01 8.4195335e-06
 7.7473078e-06 1.6563516e-07 1.4073900e-06 7.3079747e-08 3.3010316e-08
 5.6985681e-07 2.2338561e-06 1.5136418e-06 9.7920045e-09 1.3742236e-07
 3.971

### 入力テンソルの形状確認

In [14]:
# モデルのパス
tflite_model_path = tflite_save_path

# TensorFlow Lite インタープリタの初期化
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# 入力ディテールの取得
input_details = interpreter.get_input_details()
print(input_details[0]['shape'])  # 入力テンソルの形状を表示

[ 1 30 40]


## アンサンブル学習

In [15]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D, LeakyReLU
from keras.regularizers import l2

# モデルのリスト
models = []

# バギングの回数
n_models = 10

for _ in range(n_models):

    X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.8, random_state=42)
    y_train = y_train[:, -1, :]
    y_test = y_test[:, -1, :]

    # モデル
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_sequence, n_features)),
        MaxPooling1D(pool_size=2),
        LSTM(512, return_sequences=False, kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(256, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(128, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(64, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.001)),
        Dropout(0.2),
        Dense(n_classes, activation='softmax')
    ])

    # モデルのコンパイル
    model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

    # モデルチェックポイントのコールバック
    cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path, verbose=1, save_weights_only=False)

    # 早期打ち切り用コールバック
    es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

    # モデルのサマリー表示
    #model.summary()

    # 訓練
    model.fit(
        X_train,
        y_train,
        epochs=1000,
        batch_size=128,
        validation_data=(X_test, y_test),
        callbacks=[cp_callback, es_callback]
    )

    # モデルをリストに追加
    models.append(model)

Epoch 1/1000
5/5 [==============================] - ETA: 0s - loss: 5.0602 - accuracy: 0.0236
Epoch 1: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/gesture_classifier.hdf5
5/5 [==============================] - 5s 367ms/step - loss: 5.0602 - accuracy: 0.0236 - val_loss: 4.9339 - val_accuracy: 0.0432
Epoch 2/1000
5/5 [==============================] - ETA: 0s - loss: 4.8483 - accuracy: 0.0580
Epoch 2: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/gesture_classifier.hdf5
5/5 [==============================] - 1s 116ms/step - loss: 4.8483 - accuracy: 0.0580 - val_loss: 4.8179 - val_accuracy: 0.0647
Epoch 3/1000
5/5 [==============================] - ETA: 0s - loss: 4.6993 - accuracy: 0.0797
Epoch 3: saving model to /content/drive/MyDrive/Colab Notebooks/JSL/model/gesture_classifier.hdf5
5/5 [==============================] - 1s 122ms/step - loss: 4.6993 - accuracy: 0.0797 - val_loss: 4.6909 - val_accuracy: 0.0576
Epoch 4/1000
5/5 [===================

In [19]:
from sklearn.metrics import accuracy_score

def evaluate_models(models, X_test, y_test):

    accuracies = []

    for model in models:

        # モデルの予測
        y_pred = model.predict(X_test)
        y_test_classes = np.argmax(y_test, axis=1)
        y_pred_classes = np.argmax(y_pred, axis=1)

        # 精度の計算
        accuracy = accuracy_score(y_test_classes, y_pred_classes)
        accuracies.append(accuracy)


    # 各モデルの精度の平均を計算
    average_accuracy = np.mean(accuracies)
    return accuracies, average_accuracy


_, X_test_full, _, y_test_full = train_test_split(X_dataset, y_dataset, test_size=0.3, random_state=42)
y_test_full = y_test_full[:, -1, :]

# モデルの評価
accuracies, average_accuracy = evaluate_models(models, X_test_full, y_test_full)
print("各モデルの精度:", accuracies)
print("平均精度:", average_accuracy)

7/7 [==============================] - 0s 15ms/step
各モデルの精度: [0.6442307692307693, 0.6538461538461539, 0.7115384615384616, 0.6826923076923077, 0.7115384615384616, 0.7403846153846154, 0.7259615384615384, 0.7355769230769231, 0.6826923076923077, 0.6586538461538461]
平均精度: 0.6947115384615385


In [20]:
for i, model in enumerate(models):
    model.save(f'/content/drive/MyDrive/Colab Notebooks/JSL/model/ensemble_model_{i}.h5')

In [21]:
from joblib import dump

emsemble_model_save_path = '/content/drive/MyDrive/Colab Notebooks/JSL/model/ensemble_model.joblib'

# アンサンブルクラスのインスタンスを保存
dump(models, emsemble_model_save_path)

['/content/drive/MyDrive/Colab Notebooks/JSL/model/ensemble_model.joblib']